### Ver 1


In [11]:
import ray
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining

# Initialize Ray
ray.init(ignore_reinit_error=True)

def bbc(x, y):
    return 1 - x**2 + 1 - (y - 1)**2

def train_bbc(config):
    try:
        # Get initial values from config
        x = config["x"]
        y = config["y"]
        
        for step in range(10):
            # Evaluate the function with current parameters
            result = bbc(x, y)
            print(f"[Step {step+1}] x={x:.4f}, y={y:.4f}, result={result:.4f}")
            
            # Report the result to Ray Tune - using 'metrics' dict instead of keyword arguments
            tune.report(metrics={"objective": result, "iteration": step + 1})
    except Exception as e:
        import traceback
        print("Error occurred in train_bbc:")
        traceback.print_exc()
        raise e

pbt = PopulationBasedTraining(
    time_attr="iteration",  # Changed from "training_iteration" to match what we report
    metric="objective",
    mode="max",
    perturbation_interval=3,
    hyperparam_mutations={
        "x": lambda: tune.uniform(-3, 3).sample(),
        "y": lambda: tune.uniform(-3, 3).sample(),
    }
)

analysis = tune.run(
    train_bbc,
    name="pbt_bbc_debug",
    scheduler=pbt,
    num_samples=8,
    stop={"iteration": 10},  # Changed from "training_iteration" to match what we report
    config={
        "x": tune.uniform(-3, 3),
        "y": tune.uniform(-3, 3),
    },
    resources_per_trial={"cpu": 1},
    verbose=2,
    max_failures=3,
)

# Get and print the best configuration
best_config = analysis.get_best_config(metric="objective", mode="max")
best_trial = analysis.get_best_trial(metric="objective", mode="max")
print("\nBest (x, y):", best_config)
print("Best objective value:", best_trial.last_result["objective"])
print(f"Expected optimum: x=0, y=1, value=2")

2025-04-21 12:10:26,588	INFO worker.py:1684 -- Calling ray.init() again after it has already been called.
2025-04-21 12:10:26,600	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


Trial name,iteration,objective
train_bbc_efbf7_00000,10,-4.15188
train_bbc_efbf7_00001,10,1.63811
train_bbc_efbf7_00002,10,-3.15778
train_bbc_efbf7_00003,10,-7.17032
train_bbc_efbf7_00004,10,1.62473
train_bbc_efbf7_00005,10,0.394244
train_bbc_efbf7_00006,10,0.534014
train_bbc_efbf7_00007,10,-4.97424


2025-04-21 12:10:33,978	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_bbc_efbf7_00001. Skip exploit for Trial train_bbc_efbf7_00000
2025-04-21 12:10:34,321	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_bbc_efbf7_00001. Skip exploit for Trial train_bbc_efbf7_00003
2025-04-21 12:10:34,697	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_bbc_efbf7_00004. Skip exploit for Trial train_bbc_efbf7_00007
2025-04-21 12:10:47,636	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_bbc_efbf7_00004. Skip exploit for Trial train_bbc_efbf7_00003
2025-04-21 12:10:58,696	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_bbc_efbf7_00001. Skip exploit for Trial train_bbc_efbf7_00007
2025-04-21 12:11:09,655	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_bbc_efbf7_00004. Skip exploit for Trial train_bbc_efbf7_00003
2025-04-21 12:11:20,451	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_bbc_efbf7_00004. Skip exploit for Trial train_bbc_efbf7_00007
2025-0


Best (x, y): {'x': 0.4496097613929204, 'y': 1.399678980599174}
Best objective value: 1.6381077749274062
Expected optimum: x=0, y=1, value=2


### Ver 2

In [20]:
import ray
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining
# Remove problematic TensorBoard imports
import os

# Initialize Ray
ray.init(ignore_reinit_error=True)

def bbc(x, y):
    return 1 - x**2 + 1 - (y - 1)**2

def train_bbc(config):
    try:
        # Get initial values from config
        x = config["x"]
        y = config["y"]
        
        for step in range(60):
            # Evaluate the function with current parameters
            result = bbc(x, y)
            print(f"[Step {step+1}] x={x:.4f}, y={y:.4f}, result={result:.4f}")
            # Report the result to Ray Tune - using 'metrics' dict instead of keyword arguments
            tune.report(metrics={"objective": result, "iteration": step + 1})

    except Exception as e:
        import traceback
        print("Error occurred in train_bbc:")
        traceback.print_exc()
        raise e

pbt = PopulationBasedTraining(
    time_attr="training_iteration",  # Use standard Ray Tune iteration counter
    metric="objective",
    mode="max",
    perturbation_interval=3, 
    hyperparam_mutations={
        "x": lambda: tune.uniform(-3, 3).sample(),
        "y": lambda: tune.uniform(-3, 3).sample(),
    }
)

# Specify a unique experiment name for easier identification in TensorBoard
experiment_name = "pbt_bbc_local"

# Get absolute path to RL2Grid directory
current_dir = os.path.abspath(".")  # Use current directory instead of __file__
results_dir = os.path.join(current_dir, "ray_results")

# Make sure directory exists
os.makedirs(results_dir, exist_ok=True)

analysis = tune.run(
    train_bbc,
    name=experiment_name,
    storage_path=results_dir,  # Using storage_path instead of local_dir
    scheduler=pbt,
    num_samples=16,
    stop={"done": True},  # 
    config={
        "x": tune.uniform(-3, 3),
        "y": tune.uniform(-3, 3),
    },
    resources_per_trial={"cpu": 1},
    verbose=2,
    max_failures=3,
)

# Get and print the best configuration
best_config = analysis.get_best_config(metric="objective", mode="max")
best_trial = analysis.get_best_trial(metric="objective", mode="max")
print("\nBest (x, y):", best_config)
print("Best objective value:", best_trial.last_result["objective"])
print(f"Expected optimum: x=0, y=1, value=2")

# Print TensorBoard instructions with the correct local path
log_dir = os.path.join(results_dir, experiment_name)
print("\n=== TensorBoard Visualization ===")
print(f"TensorBoard logs are saved to: {log_dir}")
print("To view results in TensorBoard, run the following command in your terminal:")
print(f"tensorboard --logdir={log_dir}")
print("\nOr if you're in a Jupyter notebook, you can run:")
print("# Install TensorBoard if needed")
print("# !pip install tensorboard")
print("from tensorboard import notebook")
print(f"notebook.start('--logdir={log_dir}')")

# Additional note about Ray and TensorBoard
print("\nNote: Ray Tune automatically logs metrics to TensorBoard format")
print("even without special TensorBoard loggers or callbacks.")

2025-04-21 13:46:58,659	INFO worker.py:1684 -- Calling ray.init() again after it has already been called.
2025-04-21 13:46:58,662	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


Trial name,iteration,objective
train_bbc_6c169_00000,60,-6.92293
train_bbc_6c169_00001,60,-0.608323
train_bbc_6c169_00002,60,-1.21567
train_bbc_6c169_00003,60,1.11561
train_bbc_6c169_00004,60,-9.3452
train_bbc_6c169_00005,60,-9.64694
train_bbc_6c169_00006,60,-3.39742
train_bbc_6c169_00007,60,0.768614
train_bbc_6c169_00008,60,-2.86
train_bbc_6c169_00009,60,-2.66632


2025-04-21 13:47:05,829	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_bbc_6c169_00004. Skip exploit for Trial train_bbc_6c169_00005
2025-04-21 13:47:16,261	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_bbc_6c169_00001. Skip exploit for Trial train_bbc_6c169_00013
2025-04-21 13:47:16,554	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_bbc_6c169_00001. Skip exploit for Trial train_bbc_6c169_00012
2025-04-21 13:47:33,389	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_bbc_6c169_00001. Skip exploit for Trial train_bbc_6c169_00004
2025-04-21 13:47:36,169	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_bbc_6c169_00001. Skip exploit for Trial train_bbc_6c169_00005
2025-04-21 13:47:56,399	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_bbc_6c169_00010. Skip exploit for Trial train_bbc_6c169_00012
2025-04-21 13:47:59,229	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_bbc_6c169_00007. Skip exploit for Trial train_bbc_6c169_00013
2025-0


Best (x, y): {'x': -0.9358160756802087, 'y': 1.0929183533097753}
Best objective value: 1.1156144521166935
Expected optimum: x=0, y=1, value=2

=== TensorBoard Visualization ===
TensorBoard logs are saved to: c:\Users\admin\Grid2Op\RL2Grid_Template\RL2Grid\ray_results\pbt_bbc_local
To view results in TensorBoard, run the following command in your terminal:
tensorboard --logdir=c:\Users\admin\Grid2Op\RL2Grid_Template\RL2Grid\ray_results\pbt_bbc_local

Or if you're in a Jupyter notebook, you can run:
# Install TensorBoard if needed
# !pip install tensorboard
from tensorboard import notebook
notebook.start('--logdir=c:\Users\admin\Grid2Op\RL2Grid_Template\RL2Grid\ray_results\pbt_bbc_local')

Note: Ray Tune automatically logs metrics to TensorBoard format
even without special TensorBoard loggers or callbacks.
